In [1]:
import numpy as np
import pandas as pd
import dateutil

In [2]:
df = pd.read_csv(r'test_data\tiny_44147_B.ASC', header=None, skiprows=20,  
                             infer_datetime_format=True, encoding='ANSI',
                             engine='python', parse_dates=[0], skipinitialspace=True,
                             sep=None)

In [3]:
df

,0,1,2,3,4
0,2007-06-30,19:46:49,196.20521,412.475,133.288
1,2007-06-30,19:46:59,196.20799,412.476,133.288
2,2007-06-30,19:47:09,196.21077,412.476,133.288
3,2007-06-30,19:47:19,196.21354,412.476,133.288
4,2007-06-30,19:47:29,196.21632,412.477,133.288
...,...,...,...,...,...
3516,2007-01-07,05:32:46,205.97108,413.391,133.313
3517,2007-01-07,05:32:56,205.97386,413.392,133.313
3518,2007-01-07,05:33:06,205.97664,413.392,133.313
3519,2007-01-07,05:33:16,205.97941,413.392,133.313


In [4]:
df[0]

0      2007-06-30
1      2007-06-30
2      2007-06-30
3      2007-06-30
4      2007-06-30
          ...    
3516   2007-01-07
3517   2007-01-07
3518   2007-01-07
3519   2007-01-07
3520   2007-01-07
Name: 0, Length: 3521, dtype: datetime64[ns]

In [8]:
date_as_str = df[0].dt.strftime('%d-%m-%y')
date_as_str

0       30-06-07
1       30-06-07
2       30-06-07
3       30-06-07
4       30-06-07
          ...   
3516    07-01-07
3517    07-01-07
3518    07-01-07
3519    07-01-07
3520    07-01-07
Name: 0, Length: 3521, dtype: object

In [12]:
date_inv=pd.to_datetime(date_as_str, format='%m-%d-%y', errors='coerce')

In [15]:
date_inv = date_inv.fillna(df[0])

In [49]:
def _check_dates(ds):
    dis = ds+np.timedelta64(1, "D")
    dis = dis.shift(1)
    dck = (dis >= ds)
    dck[0] = True # because of the shift
    nwrong = (~dck).sum()

    dis = ds-np.timedelta64(1, "D")
    dis = dis.shift(1)
    dck = (dis < ds)
    dck[0] = True # because of the shift
    nwrong += (~dck).sum()
    return nwrong

In [60]:
def _try_day_month_swap(ds):
    date_as_str = ds.dt.strftime('%d-%m-%y %H:%M:%S.%f')
    date_inv=pd.to_datetime(date_as_str, format='%m-%d-%y %H:%M:%S.%f', errors='coerce')
    date_inv = date_inv.fillna(ds)
    
    n1 = _check_dates(ds)
    n2 = _check_dates(date_inv)
    if (n1<=n2):
        print("Kept day/month the same")
        return ds
    else:
        print("Swapped day/month")
        return date_inv

In [61]:
_try_day_month_swap(df[0])

Swapped day/month


0      2007-06-30
1      2007-06-30
2      2007-06-30
3      2007-06-30
4      2007-06-30
          ...    
3516   2007-07-01
3517   2007-07-01
3518   2007-07-01
3519   2007-07-01
3520   2007-07-01
Name: 0, Length: 3521, dtype: datetime64[ns]

In [62]:
df2 = pd.read_csv(r'test_data\tiny_test.cli', header=None, skiprows=20,  
                             infer_datetime_format=True, encoding='ANSI',
                             engine='python', parse_dates=[0], skipinitialspace=True,
                             sep=None)

In [63]:
df2

,0,1,2
0,2022-05-05 16:32:46,1.021174,23.012627
1,2022-05-05 16:32:47,1.021168,23.011930
2,2022-05-05 16:32:48,1.021230,23.011249
3,2022-05-05 16:32:49,1.021246,23.010487
4,2022-05-05 16:32:50,1.021287,23.009835
...,...,...,...
405,2022-05-05 16:39:31,0.974018,23.710824
406,2022-05-05 16:39:32,0.973842,23.711551
407,2022-05-05 16:39:33,0.973218,23.712359
408,2022-05-05 16:39:34,0.971580,23.713123


In [64]:
_try_day_month_swap(df2[0])

Kept day/month the same


0     2022-05-05 16:32:46
1     2022-05-05 16:32:47
2     2022-05-05 16:32:48
3     2022-05-05 16:32:49
4     2022-05-05 16:32:50
              ...        
405   2022-05-05 16:39:31
406   2022-05-05 16:39:32
407   2022-05-05 16:39:33
408   2022-05-05 16:39:34
409   2022-05-05 16:39:35
Name: 0, Length: 410, dtype: datetime64[ns]